# 휴대폰 가격 예측 보고서 (파이프라인 파일)

# 1. Spark세션 불러오기

In [1]:
from pyspark.sql import SparkSession

In [2]:
# 메모리 부족하지 않도록 용량 8g로 설정

MAX_MEMORY = '8g'

In [3]:
spark = SparkSession.builder.appName("MobilePhonePricePredict")\
    .config('spark.driver.memory', MAX_MEMORY)\
    .config('spark.executor.memory', MAX_MEMORY)\
    .getOrCreate()

## 1-1. 데이터 가져오기

In [4]:
import os

data_file_path = 'mobile_dataset.csv'
df = spark.read.csv(data_file_path, inferSchema=True, header=True)

# 2. 모델 만들기

In [5]:
train_df, test_df = df.randomSplit([0.7,0.3], seed=42)   # 데이터프레임을 7:3 비율로 분할

# 3. 전처리 파이프라인 구축

In [6]:
# 전처리할 범주형 특성 목록
# EDA에서 중요하다고 판단한 'processor', 'storage', 'battery', 'camera'
cat_features = ['processor', 'storage', 'battery', 'camera']

# 전처리할 숫자형 특성 목록
# EDA에서 중요하다고 판단한'Spec Score'와 'rating'
num_features = ['Spec Score', 'rating']

## 3-1. 범주형 데이터 전처리 파이프라인 구축

In [7]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [8]:
stages = []

In [9]:
for cat in cat_features:
    cat_index = StringIndexer(inputCol=cat, outputCol=cat+'_idx').setHandleInvalid('keep')
    onehot_encode = OneHotEncoder(inputCols=[cat_index.getOutputCol()], outputCols=[cat+'_onehot'])
    
    stages += [cat_index, onehot_encode]

stages

[StringIndexer_2289fbfe42fa,
 OneHotEncoder_85287a3b723f,
 StringIndexer_40694006918d,
 OneHotEncoder_e988ea507355,
 StringIndexer_3558e4d153f7,
 OneHotEncoder_141b2e206f8f,
 StringIndexer_8976c7b4d4ad,
 OneHotEncoder_0757e7df7f0c]

## 3-2. 숫자형 데이터 전처리 파이프라인 구축

In [10]:
from pyspark.ml.feature import StandardScaler, VectorAssembler

In [11]:
for num in num_features:
    num_assembler = VectorAssembler(inputCols=[num], outputCol=num+'_vector')
    num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(), outputCol=num+'_scaled')
    
    stages += [num_assembler, num_scaler]

stages

[StringIndexer_2289fbfe42fa,
 OneHotEncoder_85287a3b723f,
 StringIndexer_40694006918d,
 OneHotEncoder_e988ea507355,
 StringIndexer_3558e4d153f7,
 OneHotEncoder_141b2e206f8f,
 StringIndexer_8976c7b4d4ad,
 OneHotEncoder_0757e7df7f0c,
 VectorAssembler_41d9cb8c4d31,
 StandardScaler_27c7b1e11952,
 VectorAssembler_f8f064e2cbaa,
 StandardScaler_633e1c9c6195]

In [12]:
assembler_input = [cat+'_onehot' for cat in cat_features] + [num+'_scaled' for num in num_features]
assembler_input

['processor_onehot',
 'storage_onehot',
 'battery_onehot',
 'camera_onehot',
 'Spec Score_scaled',
 'rating_scaled']

In [13]:
assembler = VectorAssembler(inputCols=assembler_input, outputCol='feature_vector')
stages += [assembler]
stages

[StringIndexer_2289fbfe42fa,
 OneHotEncoder_85287a3b723f,
 StringIndexer_40694006918d,
 OneHotEncoder_e988ea507355,
 StringIndexer_3558e4d153f7,
 OneHotEncoder_141b2e206f8f,
 StringIndexer_8976c7b4d4ad,
 OneHotEncoder_0757e7df7f0c,
 VectorAssembler_41d9cb8c4d31,
 StandardScaler_27c7b1e11952,
 VectorAssembler_f8f064e2cbaa,
 StandardScaler_633e1c9c6195,
 VectorAssembler_4ccee8d2e09f]

## 3-3. 파이프라인 만들고 학습 데이터에 적용

In [14]:
from pyspark.ml import Pipeline

In [15]:
pipeline = Pipeline(stages=stages)

fitted_transform = pipeline.fit(train_df)
vtrain_df = fitted_transform.transform(train_df)

vtrain_df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Spec Score: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- price: integer (nullable = true)
 |-- img: string (nullable = true)
 |-- tag: string (nullable = true)
 |-- sim: string (nullable = true)
 |-- processor: string (nullable = true)
 |-- storage: string (nullable = true)
 |-- battery: string (nullable = true)
 |-- display: string (nullable = true)
 |-- camera: string (nullable = true)
 |-- memoryExternal: string (nullable = true)
 |-- version: string (nullable = true)
 |-- fm: string (nullable = true)
 |-- processor_idx: double (nullable = false)
 |-- processor_onehot: vector (nullable = true)
 |-- storage_idx: double (nullable = false)
 |-- storage_onehot: vector (nullable = true)
 |-- battery_idx: double (nullable = false)
 |-- battery_onehot: vector (nullable = true)
 |-- camera_idx: double (nullable = false)
 |-- camera_onehot: vector (nullable = true)
 |-- Spec Score_vector: vector (nullable = true)
 |-- Spec

In [16]:
vtrain_df.select('feature_vector').show(5)

+--------------------+
|      feature_vector|
+--------------------+
|(718,[1,207,234,4...|
|(718,[1,207,249,4...|
|(718,[0,210,234,5...|
|(718,[0,207,234,5...|
|(718,[35,209,234,...|
+--------------------+
only showing top 5 rows



# 4. 모델 만들기 및 학습

## 4-1. LinearRegression

In [17]:
from pyspark.ml.regression import LinearRegression

In [18]:
lr = LinearRegression(maxIter=50, solver='normal', labelCol='price', featuresCol='feature_vector')

In [19]:
model = lr.fit(vtrain_df)

In [20]:
vtest_df = fitted_transform.transform(test_df)

pred = model.transform(vtest_df)

In [21]:
pred.cache()

DataFrame[Name: string, Spec Score: int, rating: double, price: int, img: string, tag: string, sim: string, processor: string, storage: string, battery: string, display: string, camera: string, memoryExternal: string, version: string, fm: string, processor_idx: double, processor_onehot: vector, storage_idx: double, storage_onehot: vector, battery_idx: double, battery_onehot: vector, camera_idx: double, camera_onehot: vector, Spec Score_vector: vector, Spec Score_scaled: vector, rating_vector: vector, rating_scaled: vector, feature_vector: vector, prediction: double]

In [22]:
pred.select('price', 'prediction').show(5)

+-----+------------------+
|price|        prediction|
+-----+------------------+
|10999|11234.325588128015|
|19999| 21830.60515727061|
|43900| 43421.12665071717|
|48900| 44643.53233537533|
|60900| 47699.22207947068|
+-----+------------------+
only showing top 5 rows



## 4-2. LinearRegression 모델 평가

In [23]:
# 훈련 데이터에 대한 요약 정보 r2 & RMSE
lr_r2 = model.summary.r2
lr_RMSE = model.summary.rootMeanSquaredError
print(f'LinearRegresstion모델의 r2 : {lr_r2:.4f}')
print(f'LinearRegresstion모델의 RMSE : {lr_RMSE:.4f}')

LinearRegresstion모델의 r2 : 0.9950
LinearRegresstion모델의 RMSE : 2964.4374


In [24]:
# 테스트 데이터에 대한 평가 r2 & RMSE
from pyspark.ml.evaluation import RegressionEvaluator

# R-squared(결정계수) 평가
lr_evaluator_r2 = RegressionEvaluator(
    labelCol='price',
    predictionCol='prediction',
    metricName='r2'
)
lr_r2_score = lr_evaluator_r2.evaluate(pred)

# RMSE(제곱근 평균 제곱 오차) 평가
lr_evaluator_rmse = RegressionEvaluator(
    labelCol='price',
    predictionCol='prediction',
    metricName='rmse'
)
lr_rmse_score = lr_evaluator_rmse.evaluate(pred)

print(f'LinearRegresstion모델의 r2 : {lr_r2:.4f}')
print(f'LinearRegresstion모델의 RMSE : {lr_RMSE:.4f}')

LinearRegresstion모델의 r2 : 0.9950
LinearRegresstion모델의 RMSE : 2964.4374


# 5. 결론

+ 결정계수 (R2): 0.9950

+ 제곱근 평균 제곱 오차 (RMSE): 2964.44

탐색적 데이터 분석(EDA)을 통해 얻은 인사이트를 바탕으로, 휴대폰의 사양 정보(프로세서, 저장 용량, 스펙 점수 등)가 가격에 미치는 영향을 분석하여 선형 회귀 모델을 구축했습니다.

모델 평가 결과, `매우 높은 R2 값(0.9950)`과 `낮은 RMSE 값(2964.44)`을 얻었습니다. 이는 우리가 구축한 모델이 **휴대폰 가격을 정확하게 예측하는 데 매우 효과적**이라는 것을 의미합니다. <br>
특히 Spec Score와 같은 **숫자형 특성들이 가격 예측에 결정적인 역할**을 한 것으로 분석됩니다.

따라서, 이 모델은 향후 휴대폰 가격을 예측하거나, 새로운 휴대폰 모델의 가격을 책정하는 데 유용한 도구가 될 수 있습니다.

In [ ]:
spark.stop()